In [100]:
from collections import defaultdict
import copy

import pint
from pint import UnitRegistry
units = UnitRegistry()
Q_ = units.Quantity

import sigfig

import autoprotocol
import json
from autoprotocol.protocol import Protocol
from autoprotocol.container import Well, WellGroup, Container
from autoprotocol.container_type import ContainerType
from autoprotocol.unit import Unit


def verify_amount(amount, total=None, maximum=None):
    
    """"checks validity of quantity and unit input when adding/removing/transferring reagents
    if valid returns standardized amount w/units in proper Pint format"""
    
    #note: if amount is not numerical, error message is automatically raised in attempting to compare mag to 0
    #note: units will be automatically checked against unit registry 
        
    if amount <=0:
        raise ValueError("Cannot transfer a negative amount")
    
    elif total!=None: #specific to remove method only
        if total==0:
            raise ValueError("Cannot transfer from an empty container (or one that contains no liquid)")
            
        elif amount>total:
            raise ValueError("Quantity to be transferred should be less than total quantity in container")
        
        else:
            return amount
    
    elif maximum!=None: #if max volume of container is specified 
            if amount>maximum:
                raise ValueError("Quantity to be transferred exceeds maximum container volume")
            else:
                return amount
    else:
        return amount
        
def round_amount(amount, sigfigs=3):
    
    """"" rounds amount (which must be in proper Pint format) to a default of 3 significant figures"""
    
    mag=sigfig.round(amount.magnitude, sigfigs)
    units=amount.units
    
    return(Q_(mag, units))


"""Classes modelling chemicals and substances
"""

"""
material - solid, liquid, or gas reagent from which we can make a mixture.
stored with its properties (molar mass and density would probably be useful) and phase (state of matter)
"""

class MaterialModel():
    
    def __init__(self, name, phase, category, properties=None): 
        
        self.name = name ##we should standardize the input format for names eventually
        
        self.properties = properties #optional... can include density, molar mass, etc
        
        # phase is a state of matter (solid, liquid, gas) ... 
        #to distinguish from the more common "state" of system at any given time
        self.phase = phase 
        
        self.category = category #e.g. reagent, solute, solvent, acid, base etc
        
    def __repr__(self): 
        return f'{self.name}'
    
    def __hash__(self): 
        return hash(self.name)   

    
class ContainerModel(Container):
    
    def __init__(self, container_type, name, temp = None): 
        
        Container.__init__(self, id, container_type, name)
        
        max_vol=container_type.well_volume_ul
        
        
        vol_q=float(max_vol.magnitude)
        vol_u=max_vol.units
        
        self.max_vol=Q_(vol_q, vol_u)
        
        self.wells=Container.all_wells(self)
        
        self.wellcount=container_type.well_count
        
        self.temp=temp
        
        if container_type.is_tube==False:
        
            for well in self.wells:
                well.index=well.humanize()
                well.contents=defaultdict(lambda:0)
        else:
            
            self.contents=defaultdict(lambda:0)
            
    @property
    def total_volume(self):
        
        if self.container_type.is_tube==False:
            vol=defaultdict(lambda:0)
            for well in self.wells:
                for mat, amount in well.contents.items():
                    if mat.phase=='liquid':
                        vol[well.index]+= amount
            return vol
        
        else:
            vol=Q_(0, 'mL')
            for mat, amount in self.contents.items():
                if mat.phase=='liquid':
                    vol+= amount
            return round_amount(vol)
    
    def add(self, reagent, amount, indices='all'):
        
        if reagent.phase=='liquid':
            a= verify_amount(amount, maximum=self.max_vol)
        else:
            a= verify_amount(amount)
        
        if self.container_type.is_tube==False:
            if indices=='all':
                for well in self.wells:
                    well.contents[reagent]+=a
            else:
                for index in indices:
                    for well in self.wells:
                        if index==well.index:
                            well.contents[reagent]+=a
        else:
            self.contents[reagent] += a
                         
    def __repr__(self): 
        return f'{self.name}'
    
    def __hash__(self): 
        return hash(self.name)
    
def remove(container, amount):
        
        #assumes ideal mixture and removes proportional amount of everything
        #eventually extraction can be incorporated 
    
    removed = defaultdict(lambda: 0)
        
    initial_vol = container.total_volume
            
    verify_amount(amount, total=initial_vol)
    
    for mat, val in container.contents.items():
        
        removed[mat]+= round_amount((amount * val/initial_vol).to_reduced_units())
        container.contents[mat]-= round_amount((amount * val/initial_vol).to_reduced_units())
            
    return removed
    
class Workspace():
    
    def __init__(self, containers): 
        
        self.containers=containers
        
    def state(self):
        current=[]
        for container in self.containers:
            if container.container_type.is_tube==True:
                current.append([container.name, container.contents.materials, container.temp])
                
            else:
        
                for well in container.all_wells():
                    current.append([well, well.contents.materials, well.temp])

        return current
            
    
    def __repr__(self): 
        
        conts = [c.name for c in self.containers] 
        return f"Workspace containing {', '.join(conts)}"
    
    def __hash__(self): 
        return hash(self.containers)
    
    
class Workflow():
    
    def __init__(self, workspace): 
        
        self.workspace=workspace
    
    def step(self, action):
        
        result = action.do()
        return self.workspace.state(), result
    
    
class HeatCoolAction():
    
    def __init__(self, container, f_temp, rate=None): 
        self.container = container
        self.f_temp = f_temp
        self.rate= rate
        
    
    def do(self):
        
        self.container.temp = self.f_temp
        
        return f'Heated/cooled {self.container} to {self.f_temp}'

class StirAction():
    
    def __init__(self, container, rpm, time, temp=None): 
        self.container = container
        self.rpm = rpm
        self.time = time #can also be a string indicating purpose, e.g. until dissolved
        self.temp=temp
        
    def do(self): 
        return f'Stirred {self.container} at {self.rpm} rpm for {self.time}' 

class StoreAction():
    
    def __init__(self, container, time, temp='room', cover=None, stir='no'): 
        self.container = container
        self.time = time #can also be a string indicating purpose, e.g. until ready for use
        self.temp=temp
        self.cover=cover
        self.stir=stir
        
    def do(self): 
        return f'Stored {self.container} at {self.temp} for {self.time}. Cover: {self.cover}. Stirred: {self.stir}'
    
        
class TransferAction():

    def __init__(self, container_from, container_to, amount, index=None): 
        
        self.container_from = container_from
        self.container_to = container_to
        self.amount = amount
        self.index=index

    def do(self):
        
        if self.index!=None: #if transferring into a well of a plate
            
            removed=remove(self.container_from, self.amount*self.container_to.wellcount)
            
            for mat, val in removed.items():
                
                self.container_to.add(mat, val/self.container_to.wellcount, self.index)
                    
            return f'Transfered {self.amount} from {self.container_from} to {self.index} in {self.container_to}' 
            
        else: #if transferring between two independent containers, e.g. beaker to beaker
        
            removed=remove(self.container_from, self.amount)
            for mat, val in removed.items():
                self.container_to.add(mat, val)
        
            return f'Transfered {self.amount} from {self.container_from} to {self.container_to}'
        

In [101]:
#materials
PbI2 = MaterialModel(name='lead iodide', phase= 'solid', category='solute')
NH4I = MaterialModel(name='ammonium iodide', phase= 'solid', category='solute')
GBL = MaterialModel(name='GBL', phase= 'liquid', category='solvent')
HCOOH = MaterialModel(name='formic acid', phase= 'liquid', category='solvent')

In [104]:
#container types and containers

type_well96 = ContainerType(name='96-well PCR plate', is_tube=False, well_count=96, well_depth_mm=None, well_volume_ul=Unit(1.0, 'milliliter'), well_coating=None, sterile=None, cover_types=None, seal_types=['ultra-clear', 'foil'], capabilities=['liquid_handle', 'sangerseq', 'spin', 'thermocycle', 'incubate', 'gel_separate', 'gel_purify', 'seal', 'dispense'], shortname='4-pcr', col_count=2, dead_volume_ul=Unit(3, 'microliter'), safe_min_volume_ul=Unit(5, 'microliter'))

well96= Container(id, type_well96, name='96 well plate')

type_well4 = ContainerType(name='4-well PCR plate', is_tube=False, well_count=4, well_depth_mm=None, well_volume_ul=Unit(160.0, 'microliter'), well_coating=None, sterile=None, cover_types=None, seal_types=['ultra-clear', 'foil'], capabilities=['liquid_handle', 'sangerseq', 'spin', 'thermocycle', 'incubate', 'gel_separate', 'gel_purify', 'seal', 'dispense'], shortname='4-pcr', col_count=2, dead_volume_ul=Unit(3, 'microliter'), safe_min_volume_ul=Unit(5, 'microliter'))

well4= Container(id, type_well4, name='4 well plate')

type_beaker = ContainerType(name='50 mL beaker', is_tube=True, well_count=1, well_depth_mm=None, well_volume_ul=Unit(50, 'milliliter'), well_coating=None, sterile=None, cover_types=None, seal_types=['ultra-clear', 'foil'], capabilities=['liquid_handle', 'sangerseq', 'spin', 'thermocycle', 'incubate', 'gel_separate', 'gel_purify', 'seal', 'dispense'], shortname='beaker', col_count=0, dead_volume_ul=Unit(3, 'microliter'), safe_min_volume_ul=Unit(5, 'microliter'))

beaker50=Container(id, type_beaker, name='50 mL beaker')

type_tube = ContainerType(name='centrifuge tube', is_tube=True, well_count=1, well_depth_mm=None, well_volume_ul=Unit(15, 'milliliter'), well_coating=None, sterile=None, cover_types=None, seal_types=['ultra-clear', 'foil'], capabilities=['liquid_handle', 'sangerseq', 'spin', 'thermocycle', 'incubate', 'gel_separate', 'gel_purify', 'seal', 'dispense'], shortname='beaker', col_count=0, dead_volume_ul=Unit(3, 'microliter'), safe_min_volume_ul=Unit(5, 'microliter'))

container1 = ContainerModel(container_type=type_beaker, name='reagent 1')

container2 = ContainerModel(container_type=type_tube, name='reagent 2')

container3 = ContainerModel(container_type=type_tube, name='reagent 3')

container4= ContainerModel(container_type=type_tube, name='reagent 4')

wellplate = ContainerModel(container_type=type_well96, name='96-well plate')

In [105]:
#add materials to containers - eventually there should be a "prepare mixture" action for this

container1.add(GBL, Q_(25, 'mL'))

container2.add(PbI2, Q_(0.5, 'g'))

container2.add(NH4I, Q_(0.5, 'g'))

container2.add(GBL, Q_(10, 'mL'))
               
container3.add(NH4I, Q_(1, 'g'))

container3.add(GBL, Q_(10, 'mL'))

container4.add(HCOOH, Q_(10, 'mL'))
                             
container1.contents, container2.contents, container3.contents, container4.contents

(defaultdict(<function __main__.ContainerModel.__init__.<locals>.<lambda>()>,
             {GBL: 25 <Unit('milliliter')>}),
 defaultdict(<function __main__.ContainerModel.__init__.<locals>.<lambda>()>,
             {lead iodide: 0.5 <Unit('gram')>,
              ammonium iodide: 0.5 <Unit('gram')>,
              GBL: 10 <Unit('milliliter')>}),
 defaultdict(<function __main__.ContainerModel.__init__.<locals>.<lambda>()>,
             {ammonium iodide: 1 <Unit('gram')>,
              GBL: 10 <Unit('milliliter')>}),
 defaultdict(<function __main__.ContainerModel.__init__.<locals>.<lambda>()>,
             {formic acid: 10 <Unit('milliliter')>}))

In [106]:
#stir reagents in containers 2 and 3 (need to sort out names and representations)

for container in [container2, container3]:
    action1=StirAction(container, 450, Q_(1, 'hour'), temp=Q_(75, 'degC')).do()
    print(action1)

Stirred reagent 2 at 450 rpm for 1 hour
Stirred reagent 3 at 450 rpm for 1 hour


In [107]:
#cool containers 2 and 3 to room temp

for container in [container2, container3]:
    action2=HeatCoolAction(container, f_temp=Q_(25, 'degC')).do()
    print(action2)
    print(f'temp of {container} : {container.temp}')

Heated/cooled reagent 2 to 25 degC
temp of reagent 2 : 25 degC
Heated/cooled reagent 3 to 25 degC
temp of reagent 3 : 25 degC


In [108]:
#heat well plate to 105 (actual temp is 95... how to distinguish this?)

action3=HeatCoolAction(wellplate, f_temp=Q_(105, 'degC'), rate=Q_(7, 'degC/min')).do()

print(action3)
print(f'temp of wellplate : {wellplate.temp}')

Heated/cooled 96-well plate to 105 degC
temp of wellplate : 105 degC


In [109]:
#add 1 microliter of reagent 1 to each of the plates 

action5 = TransferAction(container1, wellplate, Q_(1, 'uL'), index='all').do()

print(action5)
print(container1.contents)
for well in wellplate.all_wells():
    
    print(well.contents)

Transfered 1 microliter from reagent 1 to all in 96-well plate
defaultdict(<function ContainerModel.__init__.<locals>.<lambda> at 0x121152560>, {GBL: <Quantity(24.904, 'milliliter')>})
defaultdict(<function ContainerModel.__init__.<locals>.<lambda> at 0x1212c7440>, {GBL: <Quantity(1.0, 'microliter')>})
defaultdict(<function ContainerModel.__init__.<locals>.<lambda> at 0x1212c7290>, {GBL: <Quantity(1.0, 'microliter')>})
defaultdict(<function ContainerModel.__init__.<locals>.<lambda> at 0x1212c7560>, {GBL: <Quantity(1.0, 'microliter')>})
defaultdict(<function ContainerModel.__init__.<locals>.<lambda> at 0x1212c73b0>, {GBL: <Quantity(1.0, 'microliter')>})
defaultdict(<function ContainerModel.__init__.<locals>.<lambda> at 0x1212c75f0>, {GBL: <Quantity(1.0, 'microliter')>})
defaultdict(<function ContainerModel.__init__.<locals>.<lambda> at 0x1212c7680>, {GBL: <Quantity(1.0, 'microliter')>})
defaultdict(<function ContainerModel.__init__.<locals>.<lambda> at 0x1212c7710>, {GBL: <Quantity(1.0,

In [110]:
#add 1 microliter of reagent 2 to each of the plates 

action6 = TransferAction(container2, wellplate, Q_(1, 'uL'), index='all').do()

print(action6)
print(container2.contents)
for well in wellplate.all_wells():
    
    print(well.contents)

Transfered 1 microliter from reagent 2 to all in 96-well plate
defaultdict(<function ContainerModel.__init__.<locals>.<lambda> at 0x121152290>, {lead iodide: <Quantity(0.4952, 'gram')>, ammonium iodide: <Quantity(0.4952, 'gram')>, GBL: <Quantity(9.904, 'milliliter')>})
defaultdict(<function ContainerModel.__init__.<locals>.<lambda> at 0x1212c7440>, {GBL: <Quantity(2.0, 'microliter')>, lead iodide: <Quantity(4.9999999999999996e-05, 'gram')>, ammonium iodide: <Quantity(4.9999999999999996e-05, 'gram')>})
defaultdict(<function ContainerModel.__init__.<locals>.<lambda> at 0x1212c7290>, {GBL: <Quantity(2.0, 'microliter')>, lead iodide: <Quantity(4.9999999999999996e-05, 'gram')>, ammonium iodide: <Quantity(4.9999999999999996e-05, 'gram')>})
defaultdict(<function ContainerModel.__init__.<locals>.<lambda> at 0x1212c7560>, {GBL: <Quantity(2.0, 'microliter')>, lead iodide: <Quantity(4.9999999999999996e-05, 'gram')>, ammonium iodide: <Quantity(4.9999999999999996e-05, 'gram')>})
defaultdict(<functi

In [111]:
#add 1 microliter of reagent 3 to each of the plates

action7 = TransferAction(container3, wellplate, Q_(1, 'uL'), index='all').do()

print(action7)
print(container3.contents)
for well in wellplate.all_wells():
    
    print(well.contents)

Transfered 1 microliter from reagent 3 to all in 96-well plate
defaultdict(<function ContainerModel.__init__.<locals>.<lambda> at 0x1212c74d0>, {ammonium iodide: <Quantity(0.9904, 'gram')>, GBL: <Quantity(9.904, 'milliliter')>})
defaultdict(<function ContainerModel.__init__.<locals>.<lambda> at 0x1212c7440>, {GBL: <Quantity(3.0, 'microliter')>, lead iodide: <Quantity(4.9999999999999996e-05, 'gram')>, ammonium iodide: <Quantity(0.00015, 'gram')>})
defaultdict(<function ContainerModel.__init__.<locals>.<lambda> at 0x1212c7290>, {GBL: <Quantity(3.0, 'microliter')>, lead iodide: <Quantity(4.9999999999999996e-05, 'gram')>, ammonium iodide: <Quantity(0.00015, 'gram')>})
defaultdict(<function ContainerModel.__init__.<locals>.<lambda> at 0x1212c7560>, {GBL: <Quantity(3.0, 'microliter')>, lead iodide: <Quantity(4.9999999999999996e-05, 'gram')>, ammonium iodide: <Quantity(0.00015, 'gram')>})
defaultdict(<function ContainerModel.__init__.<locals>.<lambda> at 0x1212c73b0>, {GBL: <Quantity(3.0, 'mi

In [112]:
#add .6 microliter of reagent 4 to each of the plates

action8 = TransferAction(container4, wellplate, Q_(0.6, 'uL'), index='all').do()

print(action8)
print(container4.contents)
for well in wellplate.all_wells():
    
    print(well.contents)

Transfered 0.6 microliter from reagent 4 to all in 96-well plate
defaultdict(<function ContainerModel.__init__.<locals>.<lambda> at 0x1212c7320>, {formic acid: <Quantity(9.9424, 'milliliter')>})
defaultdict(<function ContainerModel.__init__.<locals>.<lambda> at 0x1212c7440>, {GBL: <Quantity(3.0, 'microliter')>, lead iodide: <Quantity(4.9999999999999996e-05, 'gram')>, ammonium iodide: <Quantity(0.00015, 'gram')>, formic acid: <Quantity(0.6, 'microliter')>})
defaultdict(<function ContainerModel.__init__.<locals>.<lambda> at 0x1212c7290>, {GBL: <Quantity(3.0, 'microliter')>, lead iodide: <Quantity(4.9999999999999996e-05, 'gram')>, ammonium iodide: <Quantity(0.00015, 'gram')>, formic acid: <Quantity(0.6, 'microliter')>})
defaultdict(<function ContainerModel.__init__.<locals>.<lambda> at 0x1212c7560>, {GBL: <Quantity(3.0, 'microliter')>, lead iodide: <Quantity(4.9999999999999996e-05, 'gram')>, ammonium iodide: <Quantity(0.00015, 'gram')>, formic acid: <Quantity(0.6, 'microliter')>})
default

In [113]:
#stir for 15 min at 750 rpm

action9=StirAction(wellplate, 750, Q_(15, 'min')).do()
print(action9)

Stirred 96-well plate at 750 rpm for 15 minute


In [114]:
#add 0.6 microliters of reagent 4 to each of the plates

action10 = TransferAction(container4, wellplate, Q_(0.6, 'uL'), index='all').do()

print(action10)
print(container4.contents)
for well in wellplate.all_wells():
    
    print(well.contents)

Transfered 0.6 microliter from reagent 4 to all in 96-well plate
defaultdict(<function ContainerModel.__init__.<locals>.<lambda> at 0x1212c7320>, {formic acid: <Quantity(9.884799999999998, 'milliliter')>})
defaultdict(<function ContainerModel.__init__.<locals>.<lambda> at 0x1212c7440>, {GBL: <Quantity(3.0, 'microliter')>, lead iodide: <Quantity(4.9999999999999996e-05, 'gram')>, ammonium iodide: <Quantity(0.00015, 'gram')>, formic acid: <Quantity(1.2, 'microliter')>})
defaultdict(<function ContainerModel.__init__.<locals>.<lambda> at 0x1212c7290>, {GBL: <Quantity(3.0, 'microliter')>, lead iodide: <Quantity(4.9999999999999996e-05, 'gram')>, ammonium iodide: <Quantity(0.00015, 'gram')>, formic acid: <Quantity(1.2, 'microliter')>})
defaultdict(<function ContainerModel.__init__.<locals>.<lambda> at 0x1212c7560>, {GBL: <Quantity(3.0, 'microliter')>, lead iodide: <Quantity(4.9999999999999996e-05, 'gram')>, ammonium iodide: <Quantity(0.00015, 'gram')>, formic acid: <Quantity(1.2, 'microliter')

In [115]:
#stir for 20 min at 750 rpm

action11=StirAction(wellplate, 750, Q_(20, 'min')).do()
print(action11)

Stirred 96-well plate at 750 rpm for 20 minute


In [117]:
#store well plate at 95 C for 2.5 hrs, no stirring

action12=StoreAction(wellplate, Q_(2.5, 'hrs'), temp=Q_(95, 'degC'), cover=None, stir='no').do()
print(action12)

Stored 96-well plate at 95 degC for 2.5 hour. Cover: None. Stirred: no
